In [48]:
from faker import Faker
from datetime import timedelta
import pandas as pd
import numpy as np
import json
import os


## Inventory Data

In [49]:
inventory_df = pd.read_csv('./data/inventory_data.csv')
inventory_df

,Date,Item_ID,Item_Type,Item_Name,Current_Stock,Min_Required,Max_Capacity,Unit_Cost,Avg_Usage_Per_Day,Restock_Lead_Time,Vendor_ID
0,2024-10-01,105,Consumable,Ventilator,1542,264,1018,4467.55,108,17,V001
1,2024-10-02,100,Equipment,Ventilator,2487,656,3556,5832.29,55,12,V001
2,2024-10-03,103,Equipment,Surgical Mask,2371,384,5562,16062.98,470,6,V001
3,2024-10-04,103,Consumable,Surgical Mask,2038,438,1131,744.10,207,15,V002
4,2024-10-05,107,Equipment,IV Drip,2410,338,1013,15426.53,158,12,V003
...,...,...,...,...,...,...,...,...,...,...,...
495,2026-02-08,108,Consumable,Ventilator,4072,776,3496,15934.20,257,6,V002
496,2026-02-09,100,Equipment,X-ray Machine,997,239,2186,5863.88,108,16,V002
497,2026-02-10,107,Equipment,Surgical Mask,4193,304,4213,90.30,419,27,V001
498,2026-02-11,106,Consumable,IV Drip,822,186,2385,5284.45,382,6,V003


In [50]:
## First, generate unique items

supplies_dict = {
    'Ventilator': {
        'item_id': 100,
        'item_type': 'Equipment',
        'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
        'unit_cost': (1500000, 3200000),
        'min_required': (3, 15),
        'max_capacity': (25, 50),
        'avg_usage_per_day': (1, 5),
        'restock_lead_time': (15, 30),
        'expiration_days': None,
    },
    'X-Ray-Machine': {
        'item_id': 101,
        'item_type': 'Equipment',
        'description': 'A diagnostic tool that uses radiation to create images of bones and internal structures.',
        'unit_cost': (2500000, 5000000),
        'min_required': (1, 5),
        'max_capacity': (10, 20),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (20, 45),
        'expiration_days': None,
    },
    'Defibrilator': {
        'item_id': 102,
        'item_type': 'Equipment',
        'description': 'A life-saving device that delivers electric shocks to restore normal heart rhythms in cardiac arrest patients.',
        'unit_cost': (300000, 750000),
        'min_required': (1, 5),
        'max_capacity': (8, 15),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (20, 40),
        'expiration_days': None,
    },
    'Wheelchair': {
        'item_id': 103,
        'item_type': 'Equipment',
        'description': 'A mobility aid designed to assist patients with limited or no ability to walk.',
        'unit_cost': (5000, 25000), # Manual vs Electric Wheelchairs
        'min_required': (5, 20),
        'max_capacity': (30, 100),
        'avg_usage_per_day': (0, 5),
        'restock_lead_time': (7, 20),
        'expiration_days': None,
    },
    'Hospital Bed': {
        'item_id': 104,
        'item_type': 'Equipment',
        'description': 'An adjustable bed designed for patient care, enhancing comfort and accessibility in medical settings.',
        'unit_cost': (12000, 60000),
        'min_required': (25, 100),
        'max_capacity': (150, 300),
        'avg_usage_per_day': (0, 3),
        'restock_lead_time': (15, 30),
        'expiration_days': None,
    },
    'ECG Machine': {
        'item_id': 105,
        'item_type': 'Equipment',
        'description': 'A device that records the heart’s electrical activity to detect irregularities and cardiac conditions.',
        'unit_cost': (100000, 400000),
        'min_required': (2, 5),
        'max_capacity': (10, 15),
        'avg_usage_per_day': (0, 3),
        'restock_lead_time': (15, 30),
        'expiration_days': None,
    },
    'MRI Scanner': {
        'item_id': 106,
        'item_type': 'Equipment',
        'description': 'An imaging system that uses magnetic fields and radio waves to create detailed images of internal organs.',
        'unit_cost': (25000000, 50000000),
        'min_required': (1, 2),
        'max_capacity': (2, 5),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (30, 60),
        'expiration_days': None,
    },
    'Infusion Pump': {
        'item_id': 107,
        'item_type': 'Equipment',
        'description': 'A device that precisely delivers fluids, medications, or nutrients into a patient’s bloodstream.',
        'unit_cost': (80000, 250000),
        'min_required': (3, 10),
        'max_capacity': (15, 30),
        'avg_usage_per_day': (5, 15),
        'restock_lead_time': (10, 20),
        'expiration_days': None,
    },
    'Gloves': {
        'item_id': 200,
        'item_type': 'Consumable',
        'description': 'Protective hand coverings used by medical staff to maintain hygiene and prevent contamination.',
        'unit_cost': (5, 15),
        'min_required': (1000, 3000),
        'max_capacity': (5000, 10000),
        'avg_usage_per_day': (200, 1000),
        'restock_lead_time': (5, 12),
        'expiration_days': 1825,
    },
    'IV Drip': {
        'item_id': 201,
        'item_type': 'Consumable',
        'description': 'A method of delivering fluids, nutrients, or medications directly into a patient\'s veins.',
        'unit_cost': (120, 350),
        'min_required': (300, 1000),
        'max_capacity': (2000, 5000),
        'avg_usage_per_day': (50, 200),
        'restock_lead_time': (5, 10),
        'expiration_days': 1825,
    },
    'Surgical Mask': {
        'item_id': 202,
        'item_type': 'Consumable',
        'description': 'A disposable face mask that protects against airborne contaminants and reduces infection spread.',
        'unit_cost': (3, 8),
        'min_required': (3000, 10000),
        'max_capacity': (15000, 50000),
        'avg_usage_per_day': (500, 3000),
        'restock_lead_time': (3, 10),
        'expiration_days': 1825,
    },
    'Gown': {
        'item_id': 203,
        'item_type': 'Consumable',
        'description': 'A protective garment worn by medical staff to prevent contamination and ensure a sterile environment.',
        'unit_cost': (80, 250),
        'min_required': (500, 2000),
        'max_capacity': (3000, 8000),
        'avg_usage_per_day': (100, 500),
        'restock_lead_time': (5, 12),
        'expiration_days': 1825,
    },
    'Face Shield': {
        'item_id': 204,
        'item_type': 'Consumable',
        'description': 'A transparent protective visor that guards against splashes, droplets, and airborne particles.',
        'unit_cost': (50, 120),
        'min_required': (500, 2000),
        'max_capacity': (4000, 10000),
        'avg_usage_per_day': (100, 800),
        'restock_lead_time': (3, 8),
        'expiration_days': 1825,
    },
    'Syringe': {
        'item_id': 205,
        'item_type': 'Consumable',
        'description': 'A medical instrument used to inject or withdraw fluids from the body.',
        'unit_cost': (7, 35),
        'min_required': (1500, 5000),
        'max_capacity': (8000, 20000),
        'avg_usage_per_day': (200, 2000),
        'restock_lead_time': (5, 12),
        'expiration_days': 1825,
    },
    'Bandages': {
        'item_id': 206,
        'item_type': 'Consumable',
        'description': 'Sterile materials used to cover wounds and promote healing while preventing infections.',
        'unit_cost': (10, 50),
        'min_required': (1000, 4000),
        'max_capacity': (6000, 12000),
        'avg_usage_per_day': (300, 1500),
        'restock_lead_time': (3, 10),
        'expiration_days': 1825,
    },
    'Blood Bags': {
        'item_id': 207,
        'item_type': 'Consumable',
        'description': 'Special containers designed for the collection, storage, and transfusion of blood.',
        'unit_cost': (400, 1500),
        'min_required': (300, 1000),
        'max_capacity': (1500, 5000),
        'avg_usage_per_day': (20, 150),
        'restock_lead_time': (5, 15),
        'expiration_days': 42,
    },
    'Antiseptic Solution': {
        'item_id': 208,
        'item_type': 'Consumable',
        'description': 'A liquid disinfectant used to cleanse wounds and prevent infections.',
        'unit_cost': (200, 700),
        'min_required': (500, 2000),
        'max_capacity': (3000, 8000),
        'avg_usage_per_day': (100, 500),
        'restock_lead_time': (5, 12),
        'expiration_days': 1095,
    },
    'Cotton Rolls': {
        'item_id': 209,
        'item_type': 'Consumable',
        'description': 'Soft, absorbent materials used in medical applications for wound care and cleaning.',
        'unit_cost': (50, 150),
        'min_required': (300, 1000),
        'max_capacity': (2000, 6000),
        'avg_usage_per_day': (50, 300),
        'restock_lead_time': (3, 10),
        'expiration_days': 1825,
    },
}

In [51]:
item_var_dict = {}
fake = Faker()
supplier_names = [fake.company() + ' Medical Supplies' for _ in range(5)]
contact_numbers = [fake.numerify("+63 9## ### ####") for _ in range(5)]

for key, value in supplies_dict.items():
    # Generate 5 different item variations for 5 different vendors
    item_name = key
    item_id = value['item_id']
    item_type = value['item_type']
    description = value['description']
    status = 'Active'
    min_required = value['min_required'][0]
    max_capacity = value['max_capacity'][0]

    for i in range(5):
        vendor_id = f'V{str(i+1).zfill(3)}'
        vendor_name = supplier_names[i]
        vendor_contact = contact_numbers[i]

        item_var_dict[f'{item_name}-{vendor_id}'] = {
            'item_name': item_name,
            'item_id': item_id,
            'item_type': item_type,
            'description': description,
            'status': status,
            'min_required': min_required,
            'max_capacity': max_capacity,
            'vendor_name': vendor_name,
            'vendor_contact': vendor_contact,
        }

item_var_dict

{'Ventilator-V001': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
  'status': 'Active',
  'min_required': 3,
  'max_capacity': 25,
  'vendor_name': 'Charles-Perry Medical Supplies',
  'vendor_contact': '+63 914 128 7105'},
 'Ventilator-V002': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
  'status': 'Active',
  'min_required': 3,
  'max_capacity': 25,
  'vendor_name': 'Ellis-Johnson Medical Supplies',
  'vendor_contact': '+63 975 915 2123'},
 'Ventilator-V003': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
  'status': 'Active',
  'min_required': 3,
  'max_capacity': 2

In [52]:
fake = Faker()
date_list = pd.date_range(start='2022-01-01', end='2024-12-31', freq='D').strftime('%Y-%m-%d').tolist()

# Data dictionary (to pass to df constructor)
date_arr = []
item_id_arr = []
item_name_arr = []
item_type_arr = []
description_arr = []
status_arr = []
stock_status_arr = []
unit_cost_arr = []
min_required_arr = []
max_capacity_arr = []
current_stock_arr = []
avg_usage_per_day_arr = []
restock_lead_time_arr = []
vendor_name_arr = []
vendor_contact_arr = []
expiration_date_arr = []
created_by_arr = []

for key, value in item_var_dict.items():
    # Generate around 30 - 50 random instances of a particular item
    #for _ in range(4):
    for _ in range(np.random.randint(30, 51)):
        date = fake.random_element(date_list)
        item_id = value['item_id']
        item_name = value['item_name']
        item_type = value['item_type']
        description = value['description']
        status = value['status']
        stock_status = 'Available'
        unit_cost = np.random.randint(supplies_dict[item_name]['unit_cost'][0], supplies_dict[item_name]['unit_cost'][1] + 1)
        min_required = value['min_required']
        max_capacity = value['max_capacity']
        current_stock = np.random.randint(min_required, max_capacity + 1)
        avg_usage_per_day = np.random.randint(supplies_dict[item_name]['avg_usage_per_day'][0], supplies_dict[item_name]['avg_usage_per_day'][1] + 1)
        restock_lead_time = np.random.randint(supplies_dict[item_name]['restock_lead_time'][0], supplies_dict[item_name]['restock_lead_time'][1] + 1)
        vendor_name = value['vendor_name']
        vendor_contact = value['vendor_contact']
        created_by = '67b1fdb293bfc8e17f896b31'

        if item_type == 'Consumable':
            expiration_duration = supplies_dict[item_name]['expiration_days']
            expiration_date = pd.to_datetime(date) + timedelta(days=expiration_duration)
        else:
            expiration_date = None

        date_arr.append(date)
        item_id_arr.append(item_id)
        item_name_arr.append(item_name)
        item_type_arr.append(item_type)
        description_arr.append(description)
        status_arr.append(status)
        stock_status_arr.append(stock_status)
        unit_cost_arr.append(unit_cost)
        min_required_arr.append(min_required)
        max_capacity_arr.append(max_capacity)
        current_stock_arr.append(current_stock)
        avg_usage_per_day_arr.append(avg_usage_per_day)
        restock_lead_time_arr.append(restock_lead_time)
        vendor_name_arr.append(vendor_name)
        vendor_contact_arr.append(vendor_contact)
        expiration_date_arr.append(expiration_date)
        created_by_arr.append(created_by)

data = list(zip(date_arr, item_id_arr, item_name_arr, item_type_arr, description_arr, status_arr, stock_status_arr, unit_cost_arr, min_required_arr, max_capacity_arr, 
                current_stock_arr, avg_usage_per_day_arr, restock_lead_time_arr, vendor_name_arr, vendor_contact_arr, expiration_date_arr, created_by_arr))

df = pd.DataFrame(data, columns=[
    'date', 'item_id', 'item_name', 'item_type', 'description', 'status', 'stock_status', 'unit_cost', 'min_required', 'max_capacity', 'current_stock', 'avg_usage_per_day', 'restock_lead_time', 'vendor_name', 'vendor_contact', 'expiration_date', 'created_by'
]).sort_values(by=['date']).reset_index(drop=True)

In [53]:
df

,date,item_id,item_name,item_type,description,status,stock_status,unit_cost,min_required,max_capacity,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date,created_by
0,2022-01-01,104,Hospital Bed,Equipment,"An adjustable bed designed for patient care, e...",Active,Available,49936,25,150,135,1,22,Charles-Perry Medical Supplies,+63 914 128 7105,NaT,67b1fdb293bfc8e17f896b31
1,2022-01-01,107,Infusion Pump,Equipment,"A device that precisely delivers fluids, medic...",Active,Available,224271,3,15,5,6,18,Charles-Perry Medical Supplies,+63 914 128 7105,NaT,67b1fdb293bfc8e17f896b31
2,2022-01-01,107,Infusion Pump,Equipment,"A device that precisely delivers fluids, medic...",Active,Available,160967,3,15,10,14,17,Sandoval and Sons Medical Supplies,+63 952 345 2479,NaT,67b1fdb293bfc8e17f896b31
3,2022-01-02,205,Syringe,Consumable,A medical instrument used to inject or withdra...,Active,Available,9,1500,8000,7680,662,10,Sandoval and Sons Medical Supplies,+63 952 345 2479,2027-01-01,67b1fdb293bfc8e17f896b31
4,2022-01-02,104,Hospital Bed,Equipment,"An adjustable bed designed for patient care, e...",Active,Available,36616,25,150,128,1,15,Sandoval and Sons Medical Supplies,+63 952 345 2479,NaT,67b1fdb293bfc8e17f896b31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3579,2024-12-30,202,Surgical Mask,Consumable,A disposable face mask that protects against a...,Active,Available,4,3000,15000,5700,1995,5,Mcdonald-Le Medical Supplies,+63 900 112 1712,2029-12-29,67b1fdb293bfc8e17f896b31
3580,2024-12-30,205,Syringe,Consumable,A medical instrument used to inject or withdra...,Active,Available,8,1500,8000,3117,289,9,Ellis-Johnson Medical Supplies,+63 975 915 2123,2029-12-29,67b1fdb293bfc8e17f896b31
3581,2024-12-30,205,Syringe,Consumable,A medical instrument used to inject or withdra...,Active,Available,16,1500,8000,5334,1767,11,Taylor Inc Medical Supplies,+63 972 467 2353,2029-12-29,67b1fdb293bfc8e17f896b31
3582,2024-12-31,104,Hospital Bed,Equipment,"An adjustable bed designed for patient care, e...",Active,Available,27132,25,150,64,1,27,Charles-Perry Medical Supplies,+63 914 128 7105,NaT,67b1fdb293bfc8e17f896b31


## Item Listing Data

In [55]:
item_listing_df = df[['item_id', 'item_name', 'description', 'item_type', 'min_required', 'max_capacity', 'created_by', 'status']].drop_duplicates().sort_values('item_id').reset_index(drop=True)
item_listing_df

,item_id,item_name,description,item_type,min_required,max_capacity,created_by,status
0,100,Ventilator,A medical device that supports or takes over b...,Equipment,3,25,67b1fdb293bfc8e17f896b31,Active
1,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,1,10,67b1fdb293bfc8e17f896b31,Active
2,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,1,8,67b1fdb293bfc8e17f896b31,Active
3,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,5,30,67b1fdb293bfc8e17f896b31,Active
4,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,25,150,67b1fdb293bfc8e17f896b31,Active
5,105,ECG Machine,A device that records the heart’s electrical a...,Equipment,2,10,67b1fdb293bfc8e17f896b31,Active
6,106,MRI Scanner,An imaging system that uses magnetic fields an...,Equipment,1,2,67b1fdb293bfc8e17f896b31,Active
7,107,Infusion Pump,"A device that precisely delivers fluids, medic...",Equipment,3,15,67b1fdb293bfc8e17f896b31,Active
8,200,Gloves,Protective hand coverings used by medical staf...,Consumable,1000,5000,67b1fdb293bfc8e17f896b31,Active
9,201,IV Drip,"A method of delivering fluids, nutrients, or m...",Consumable,300,2000,67b1fdb293bfc8e17f896b31,Active


In [56]:
file_name_csv = './data/item_listing_v1.csv'
file_name_json = './data/item_listing_v1.json'

if not os.path.exists(file_name_csv):
    item_listing_df.to_csv(file_name_csv, index=False)
    item_listing_df.to_json(file_name_json, orient='records')
    print('File saved successfully!')
else:
    print('File already exists!')

File already exists!


## Item Stock Listing Data

In [57]:
item_listing_df = pd.read_json('./data/exported_listings.json')
item_listing_df

,_id,itemCode,title,description,category,abcCategory,minStockLevel,maxStockLevel,createdBy,status,createdAt
0,67b5d1d3965831513a85d8be,100,Ventilator,A medical device that supports or takes over b...,Equipment,C,3,25,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
1,67b5d1d3965831513a85d8bf,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,C,1,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
2,67b5d1d3965831513a85d8c0,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,C,1,8,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
3,67b5d1d3965831513a85d8c1,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,C,5,30,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
4,67b5d1d3965831513a85d8c2,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,C,25,150,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
5,67b5d1d3965831513a85d8c3,105,ECG Machine,A device that records the heart’s electrical a...,Equipment,C,2,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
6,67b5d1d3965831513a85d8c4,106,MRI Scanner,An imaging system that uses magnetic fields an...,Equipment,C,1,2,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
7,67b5d1d3965831513a85d8c5,107,Infusion Pump,"A device that precisely delivers fluids, medic...",Equipment,C,3,15,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
8,67b5d1d3965831513a85d8c6,200,Gloves,Protective hand coverings used by medical staf...,Consumable,C,1000,5000,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
9,67b5d1d3965831513a85d8c7,201,IV Drip,"A method of delivering fluids, nutrients, or m...",Consumable,C,300,2000,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z


In [58]:
df

,date,item_id,item_name,item_type,description,status,stock_status,unit_cost,min_required,max_capacity,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date,created_by
0,2022-01-01,104,Hospital Bed,Equipment,"An adjustable bed designed for patient care, e...",Active,Available,49936,25,150,135,1,22,Charles-Perry Medical Supplies,+63 914 128 7105,NaT,67b1fdb293bfc8e17f896b31
1,2022-01-01,107,Infusion Pump,Equipment,"A device that precisely delivers fluids, medic...",Active,Available,224271,3,15,5,6,18,Charles-Perry Medical Supplies,+63 914 128 7105,NaT,67b1fdb293bfc8e17f896b31
2,2022-01-01,107,Infusion Pump,Equipment,"A device that precisely delivers fluids, medic...",Active,Available,160967,3,15,10,14,17,Sandoval and Sons Medical Supplies,+63 952 345 2479,NaT,67b1fdb293bfc8e17f896b31
3,2022-01-02,205,Syringe,Consumable,A medical instrument used to inject or withdra...,Active,Available,9,1500,8000,7680,662,10,Sandoval and Sons Medical Supplies,+63 952 345 2479,2027-01-01,67b1fdb293bfc8e17f896b31
4,2022-01-02,104,Hospital Bed,Equipment,"An adjustable bed designed for patient care, e...",Active,Available,36616,25,150,128,1,15,Sandoval and Sons Medical Supplies,+63 952 345 2479,NaT,67b1fdb293bfc8e17f896b31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3579,2024-12-30,202,Surgical Mask,Consumable,A disposable face mask that protects against a...,Active,Available,4,3000,15000,5700,1995,5,Mcdonald-Le Medical Supplies,+63 900 112 1712,2029-12-29,67b1fdb293bfc8e17f896b31
3580,2024-12-30,205,Syringe,Consumable,A medical instrument used to inject or withdra...,Active,Available,8,1500,8000,3117,289,9,Ellis-Johnson Medical Supplies,+63 975 915 2123,2029-12-29,67b1fdb293bfc8e17f896b31
3581,2024-12-30,205,Syringe,Consumable,A medical instrument used to inject or withdra...,Active,Available,16,1500,8000,5334,1767,11,Taylor Inc Medical Supplies,+63 972 467 2353,2029-12-29,67b1fdb293bfc8e17f896b31
3582,2024-12-31,104,Hospital Bed,Equipment,"An adjustable bed designed for patient care, e...",Active,Available,27132,25,150,64,1,27,Charles-Perry Medical Supplies,+63 914 128 7105,NaT,67b1fdb293bfc8e17f896b31


In [78]:
df_sub = df[['item_id', 'date', 'stock_status', 'unit_cost', 'current_stock', 'avg_usage_per_day', 'restock_lead_time', 'vendor_name', 'vendor_contact', 'expiration_date']]

stock_listing_df = item_listing_df[['_id', 'itemCode']].merge(df_sub, left_on='itemCode', right_on='item_id')
stock_listing_df.drop(columns=['itemCode', 'item_id'], inplace=True)
stock_listing_df.sort_values(by='date', inplace=True)
stock_listing_df

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date
1438,67b5d1d3965831513a85d8c5,2022-01-01,Available,224271,5,6,18,Charles-Perry Medical Supplies,+63 914 128 7105,NaT
1439,67b5d1d3965831513a85d8c5,2022-01-01,Available,160967,10,14,17,Sandoval and Sons Medical Supplies,+63 952 345 2479,NaT
822,67b5d1d3965831513a85d8c2,2022-01-01,Available,49936,135,1,22,Charles-Perry Medical Supplies,+63 914 128 7105,NaT
823,67b5d1d3965831513a85d8c2,2022-01-02,Available,36616,128,1,15,Sandoval and Sons Medical Supplies,+63 952 345 2479,NaT
2611,67b5d1d3965831513a85d8cb,2022-01-02,Available,9,7680,662,10,Sandoval and Sons Medical Supplies,+63 952 345 2479,2027-01-01
...,...,...,...,...,...,...,...,...,...,...
2213,67b5d1d3965831513a85d8c8,2024-12-30,Available,4,5700,1995,5,Mcdonald-Le Medical Supplies,+63 900 112 1712,2029-12-29
2803,67b5d1d3965831513a85d8cb,2024-12-30,Available,8,3117,289,9,Ellis-Johnson Medical Supplies,+63 975 915 2123,2029-12-29
2804,67b5d1d3965831513a85d8cb,2024-12-30,Available,16,5334,1767,11,Taylor Inc Medical Supplies,+63 972 467 2353,2029-12-29
1019,67b5d1d3965831513a85d8c2,2024-12-31,Available,27132,64,1,27,Charles-Perry Medical Supplies,+63 914 128 7105,NaT


In [79]:
file_name_json = './data/stock_listing_v1_1.json'

if not os.path.exists(file_name_json):
    stock_listing_df.to_json(file_name_json, orient='records')
    print('File saved successfully!')
else:
    print('File already exists!')

File saved successfully!


## Patient Data

In [60]:
patient_df = pd.read_csv('./data/patient_data.csv')
patient_df

,Patient_ID,Admission_Date,Discharge_Date,Primary_Diagnosis,Procedure_Performed,Room_Type,Bed_Days,Supplies_Used,Equipment_Used,Staff_Needed
0,P001,2024-10-06 05:30:28,2024-10-23 01:11:34,Diabetes,Appendectomy,General Ward,2,"Gloves, IV",Surgical Table,2 Surgeons
1,P002,2024-10-24 11:07:58,2024-10-15 05:16:54,Fracture,Appendectomy,ICU,10,"Gown, IV",MRI Machine,1 Nurse
2,P003,2024-10-22 21:43:43,2024-10-24 10:56:30,Fracture,Chest X-ray,ICU,10,"Gloves, IV",X-ray Machine,"1 Nurse, 1 Doctor"
3,P004,2024-10-05 17:04:05,2024-10-30 14:10:01,Diabetes,Chest X-ray,ICU,11,"Gloves, IV",X-ray Machine,1 Nurse
4,P005,2024-10-21 17:04:00,2024-10-08 15:53:22,Appendicitis,MRI,ICU,2,"Gloves, IV",X-ray Machine,2 Surgeons
...,...,...,...,...,...,...,...,...,...,...
495,P496,2024-10-01 12:54:33,2024-10-06 14:31:39,Diabetes,MRI,ICU,13,"Mask, Gown",Surgical Table,"1 Nurse, 1 Doctor"
496,P497,2024-10-22 07:37:10,2024-10-13 12:31:40,Appendicitis,Appendectomy,ICU,14,"Gloves, IV",MRI Machine,"1 Nurse, 1 Doctor"
497,P498,2024-10-28 03:12:20,2024-10-10 08:11:18,Pneumonia,MRI,General Ward,13,"Mask, Gown",X-ray Machine,2 Surgeons
498,P499,2024-10-04 22:02:01,2024-10-12 03:45:09,Diabetes,Chest X-ray,General Ward,1,"Gown, IV",Surgical Table,"1 Nurse, 1 Doctor"


In [61]:
# Supplies Used Item Mapping
supplies_dict = {}

for i in patient_df['Supplies_Used']:
    item_arr = i.split(', ')
    for item in item_arr:
        if item not in supplies_dict:
            supplies_dict[item] = 1
        elif item in supplies_dict:
            supplies_dict[item] += 1
        
supplies_dict

{'Gloves': 169, 'IV': 339, 'Gown': 331, 'Mask': 161}

In [62]:
# Equipment Used Item Mapping
equipment_dict = {}

for i in patient_df['Equipment_Used']:
    item_arr = i.split(', ')
    for item in item_arr:
        if item not in equipment_dict:
            equipment_dict[item] = 1
        elif item in equipment_dict:
            equipment_dict[item] += 1
        
equipment_dict

{'Surgical Table': 166, 'MRI Machine': 161, 'X-ray Machine': 173}

In [63]:
# Rename the keys in the dictionary to standardized item names
try:
    supplies_dict['IV Drip'] = supplies_dict.pop('IV')
    supplies_dict['Surgical Mask'] = supplies_dict.pop('Mask')
    equipment_dict['X-Ray-Machine'] = equipment_dict.pop('X-ray Machine')
    equipment_dict['MRI Scanner'] = equipment_dict.pop('MRI Machine')
except KeyError:
    pass

In [64]:
# EDA of 'primary_diagnosis'
patient_df['Primary_Diagnosis'].value_counts()

Primary_Diagnosis
Fracture        133
Diabetes        131
Appendicitis    120
Pneumonia       116
Name: count, dtype: int64

In [65]:
# EDA of 'procedure_performed'
patient_df['Procedure_Performed'].value_counts()

Procedure_Performed
Chest X-ray     138
Appendectomy    131
MRI             131
Blood Test      100
Name: count, dtype: int64

### Synthetic Patient Data Generation

In [66]:
diagnosis_dict = {
    'Pneumonia': {
        'Chest X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': ['Infusion Pump']
        },
        'Oxygen Therapy': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves'],
            'equipment_used': ['Ventilator']
        }
    },
    'Appendicitis': {
        'Appendectomy': {
            'supplies_used': ['Gloves', 'Gown', 'Surgical Mask', 'Antiseptic Solution', 'Bandages', 'IV Drip'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        }
    },
    'Fracture': {
        'X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        },
        'Casting': {
            'supplies_used': ['Bandages', 'Cotton Rolls', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': []
        },
        'Pain Management': {
            'supplies_used': ['Syringe', 'IV Drip', 'Gloves'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        }
    },
    'Myocardial Infarction': {
        'ECG': {
            'supplies_used': ['Gloves'],
            'equipment_used': ['ECG Machine']
        },
        'Defibrillation': {
            'supplies_used': ['Gloves', 'Surgical Mask', 'Antiseptic Solution'],
            'equipment_used': ['Defibrillator']
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Blood Bags'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Diabetes Complications': {
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Blood Bags'],
            'equipment_used': ['Infusion Pump']
        },
        'Wound Dressing': {
            'supplies_used': ['Bandages', 'Gloves', 'Antiseptic Solution', 'Cotton Rolls'],
            'equipment_used': []
        }
    },
    'Stroke': {
        'MRI': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['MRI Scanner']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'Physical Therapy': {
            'supplies_used': ['Gloves', 'Face Shield', 'Gown'],
            'equipment_used': ['Hospital Bed', 'Wheelchair']
        }
    },
    'Sepsis': {
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves', 'Blood Bags'],
            'equipment_used': []
        },
        'IV Antibiotic Therapy': {
            'supplies_used': ['IV Drip', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Severe Burns': {
        'Wound Cleaning & Dressing': {
            'supplies_used': ['Antiseptic Solution', 'Bandages', 'Gloves', 'Cotton Rolls'],
            'equipment_used': []
        },
        'Skin Grafting Surgery': {
            'supplies_used': ['Gown', 'Surgical Mask', 'Gloves'],
            'equipment_used': ['Hospital Bed']
        },
        'IV Fluid Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Traumatic Brain Injury': {
        'MRI': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['MRI Scanner']
        },
        'Ventilation Support': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves'],
            'equipment_used': ['Ventilator']
        },
        'Pain Management': {
            'supplies_used': ['Syringe', 'IV Drip', 'Gloves'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        }
    },
    'COVID-19': {
        'Oxygen Therapy': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves', 'Gown'],
            'equipment_used': ['Ventilator', 'Hospital Bed']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'Chest X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        }
    }
}

In [67]:
diagnosis_id_arr = []
date_arr = []
primary_diagnosis_arr = []
procedure_performed_arr = []
item_used_arr = []
item_type_arr = []

diagnosis_id_counter = 1

date_list = pd.date_range(start='2022-01-01', end='2024-12-31', freq='D').strftime('%Y-%m-%d').tolist()

for diagnosis in diagnosis_dict:
    for _ in range(np.random.randint(100, 500)):
        diagnosis_id = diagnosis_id_counter
        for procedure in diagnosis_dict[diagnosis]:
            for item in diagnosis_dict[diagnosis][procedure]['supplies_used']:
                diagnosis_id_arr.append(str(diagnosis_id).zfill(5))
                date_arr.append(fake.random_element(date_list))
                primary_diagnosis_arr.append(diagnosis)
                procedure_performed_arr.append(procedure)
                item_used_arr.append(item)
                item_type_arr.append('Consumable')
            for item in diagnosis_dict[diagnosis][procedure]['equipment_used']:
                diagnosis_id_arr.append(str(diagnosis_id).zfill(5))
                date_arr.append(fake.random_element(date_list))
                primary_diagnosis_arr.append(diagnosis)
                procedure_performed_arr.append(procedure)
                item_used_arr.append(item)
                item_type_arr.append('Equipment')
        diagnosis_id_counter += 1

In [68]:
print(len(diagnosis_id_arr))
print(len(primary_diagnosis_arr))
print(len(procedure_performed_arr))
print(len(item_used_arr))
print(len(item_type_arr))

30502
30502
30502
30502
30502


In [69]:
# Data dictionary (to pass to df constructor)
data = {
    'diagnosis_id': diagnosis_id_arr,
    'date': date_arr,
    'primary_diagnosis': primary_diagnosis_arr,
    'procedure_performed': procedure_performed_arr,
    'item_used': item_used_arr,
    'item_type': item_type_arr
}

patient_df_v2 = pd.DataFrame(data)
patient_df_v2

,diagnosis_id,date,primary_diagnosis,procedure_performed,item_used,item_type
0,00001,2024-05-01,Pneumonia,Chest X-Ray,Surgical Mask,Consumable
1,00001,2022-02-22,Pneumonia,Chest X-Ray,Gloves,Consumable
2,00001,2024-02-20,Pneumonia,Chest X-Ray,Gown,Consumable
3,00001,2022-07-21,Pneumonia,Chest X-Ray,Face Shield,Consumable
4,00001,2022-06-09,Pneumonia,Chest X-Ray,X-Ray Machine,Equipment
...,...,...,...,...,...,...
30497,02494,2022-09-08,COVID-19,Chest X-Ray,Surgical Mask,Consumable
30498,02494,2023-10-14,COVID-19,Chest X-Ray,Gloves,Consumable
30499,02494,2022-03-25,COVID-19,Chest X-Ray,Gown,Consumable
30500,02494,2022-10-16,COVID-19,Chest X-Ray,Face Shield,Consumable


In [70]:
file_name = './data/patient_data_v2.csv'

if not os.path.exists(file_name):
    patient_df_v2.to_csv(file_name, index=False)
    print('File saved successfully!')
else:
    print('File already exists!')

File already exists!
